In [ ]:
#Bush press briefing analysis, Jan-March 2001

In [2]:
import nltk

#did not end up needing textblob, pattern library uses same sentiment classifier
#from textblob import *
#from textblob.sentiments import NaiveBayesAnalyzer

import re
import csv

#pattern library.. simple NLP tools based on product reviews
from pattern.en import parse, Sentence, parse
from pattern.en import modality
from pattern.en import mood
from pattern.en import sentiment
from pattern.en import positive

#for grade-level score, sentence complexity
from textstat.textstat import textstat

In [3]:
#first 30 press briefings in text file, parsed into sentences using nltk.
#Note: nltk sentence parser seems to look for ". ".. does not parse on other interruptions like "--"

f = open('Bush_compiled.txt')
raw = f.read()
sents = nltk.sent_tokenize(raw)
#print sents_bush

In [ ]:
#test nlp metrics.. sentence, if_positive_sent, mood (aka sentence type), modality (aka certainty), polarity, subjectivity
#Note: TextBlob sentiment analysis, in this package, is based on pattern library, see link
#http://www.clips.ua.ac.be/pages/pattern-en#sentiment

for i in sents[:100]:
    if "FLEISCHER:" in i:
        print i, positive(i, threshold = 0.1), mood(i), modality(i), sentiment(i)[0], sentiment(i)[1]

In [4]:
#tagging nltk-produced "sentences" by speaker.. note that there were some cases where there were interjections.
#in the cases where a single "sentence" was really a single statement made up of multiple sentences split by "--", I chose to tag as being by the spokesperson.
#a subsequent script would parse by those "--" markers first, in conjunction with "Q " or "[SPOKESPERSON NAME]:", then break those statements up into individual sentences

#metrics:

    #text-based/known:
        #administration - aka "Trump", "Obama", or "Bush"
        #sentence/string
        #person: "spokesperson" or "press_corps"
        
    #from pattern library: http://www.clips.ua.ac.be/pages/pattern-en#sentiment
        #mood(string) - The mood() function returns either INDICATIVE, IMPERATIVE, CONDITIONAL or SUBJUNCTIVE for a given parsed Sentence. See the table below for an overview of moods.
        #modality(string) - The modality() function returns the degree of certainty as a value between -1.0 and +1.0, where values > +0.5 represent facts. For example, "I wish it would stop raining" scores -0.35, whereas "It will stop raining" scores +0.75. Accuracy is about 68% for Wikipedia texts.
        #sentiment: polarity & subjectivity, in that order
   
    #from textstat: 
        #flesch-kincaid grade level, based on flesch-kincaid grade formula

admin = "Bush"
person = []

with open('bush_tagged.csv', 'wb') as file:
    writer = csv.writer(file)

    #cleaning.. in some cases a Q marking a press corp question had no space btw it and the question. LGBTQ also screwed up the subsequent if statement logic
    for string in sents:
        string = re.sub(r'(Q+)([A-Z])', r'\1 \2', string)
        string = string.replace('MR.', '').replace('\r', '').replace('\n', ' ').replace('LGBTQ', 'LGBT')
       
    #tag speaker
    
        if "FLEISCHER:" in string:
            
            string = string.replace('FLEISCHER:','').lstrip()
            person = "Spokesperson"
            row = admin, string, person, positive(string, threshold = 0.1), mood(string), modality(string), sentiment(string)[0], sentiment(string)[1], textstat.flesch_kincaid_grade(string)
            #print person, [string]
            writer.writerow(row)

        elif ("Q " or "Q--" or "Q  ") in string:
        
            string = string.replace('Q ', '').replace('Q--', '').replace('Q  ', '').lstrip()
            person = "Press_Corps"
            row = admin, string, person, positive(string, threshold = 0.1), mood(string), modality(string), sentiment(string)[0], sentiment(string)[1], textstat.flesch_kincaid_grade(string)
            #print person, [string]
            writer.writerow(row)
            
    #if no tag, continue with existing until next found
    
        else:
            if len(string) != 0:
                string = string.lstrip()
                row = admin, string, person, positive(string, threshold = 0.1), mood(string), modality(string), sentiment(string)[0], sentiment(string)[1], textstat.flesch_kincaid_grade(string)
                #print person, [string]
                writer.writerow(row)
            else:
                continue
            
     

In [ ]:
#verify textblob and pattern library produce same sentiment results.. 
#had initially used textblob but pattern library came in handy for addt'l features

for i in newstring_speaker[:100]:
    if "FLEISCHER:" in i:
        blob = TextBlob(i)
        if (blob.sentiment.polarity == sentiment(i)[0]) & (blob.sentiment.subjectivity == sentiment(i)[1]):
            print "TRUE"

In [19]:
#original textblob sentiment analysis.. had not broken out by speaker at this point

speaker = ' '.join(newstring_speaker)
test = TextBlob(speaker)

with open('Bush_sentiment.csv', "wb") as file:
    writer = csv.writer(file)
    for sentence in test.sentences:
        row = sentence, sentence.sentiment.polarity, sentence.sentiment.subjectivity
        writer.writerow(row)
    
#for sentence in test.sentences[0:10]:
    #print sentence, sentence.sentiment.polarity, sentence.sentiment.subjectivity
    


In [ ]:
#textblob demo, from website

from textblob import TextBlob
>>> from textblob.sentiments import NaiveBayesAnalyzer
>>> blob = TextBlob("I love this library", analyzer=NaiveBayesAnalyzer())
>>> blob.sentiment
Sentiment(classification='pos', p_pos=0.7996209910191279, p_neg=0.2003790089808724)

In [25]:
#another example

import csv
Bush = ' '.join(newstring_bush)
blob1 = TextBlob(Bush, analyzer = NaiveBayesAnalyzer())

with open('Bush_sent_nb.csv', "wb") as file:
    writer = csv.writer(file)
    for sentence in test.sentences:
        row = sentence, sentence.sentiment.polarity, sentence.sentiment.subjectivity
        writer.writerow(row)